# 🎵 AI Music Generator

##  Imports

In [3]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
import os

## 1: Parsing and Processing MIDI Files

Extracting the note and chord data from MIDI files and convert them into string tokens.

In [54]:
notes = []
for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    print("Parsing:", file)

    elements_to_parse = midi.flat.notes

    for ele in elements_to_parse:
        if isinstance(ele, note.Note):
            notes.append(str(ele.pitch))
        elif isinstance(ele, chord.Chord):
            notes.append('+'.join(str(n) for n in ele.normalOrder))

with open("notes", 'wb') as f:
    pickle.dump(notes, f)

Parsing: midi_songs/BornToBeYoursft.Kygo(Pianoversion).mid
Parsing: midi_songs/Romanticpiano.mid


## 2: Preparing Sequential Data for LSTM

Converting the list of notes into input/output sequences that can be fed to the LSTM.

In [57]:
sequence_length = 100
n_vocab = len(set(notes))
pitchnames = sorted(set(notes))
ele_to_int = dict((ele, num) for num, ele in enumerate(pitchnames))

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length]
    seq_out = notes[i+sequence_length]
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

n_patterns = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)

## 3: Building the Model

Defining a stacked LSTM model with dropout layers for regularization.

In [60]:
model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 100, 512)       │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 63)             │        16,191 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,398,591 (20.59 MB)

 Trainable params: 5,398,591 (20.59 MB)

 Non-trainable params: 0 (0.00 B)

## 4: Training the Model

Training the model using the training data and save the best weights.

In [ ]:
checkpoint = ModelCheckpoint("model.keras", monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=[checkpoint])

## 5: Generating New Music

Loading the trained model and generating a sequence of new notes.

In [ ]:
with open("notes", 'rb') as f:
    notes = pickle.load(f)

pitchnames = sorted(set(notes))
ele_to_int = dict((ele, num) for num, ele in enumerate(pitchnames))
int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))
n_vocab = len(pitchnames)

sequence_length = 100
network_input = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length]
    network_input.append([ele_to_int[ch] for ch in seq_in])

start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

model = load_model("model.keras")

prediction_output = []

for note_index in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_ele[index]
    prediction_output.append(result)

    pattern.append(index)
    pattern = pattern[1:]

## 6: Converting Output to MIDIoffset = 0

Converting the generated note sequence into a .mid file for playback.

In [ ]:
offset = 0
output_notes = []

for pattern in prediction_output:
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
        new_chord = chord.Chord(temp_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

In [ ]:
#Play the music....
midi_stream.show('midi')